In [160]:
# this script marks already detected bad channels in the bids files
# imports
import numpy as np
import mne
import os.path as op
import pandas as pd
from mne_bids import BIDSPath, read_raw_bids, mark_bad_channels
import ast

# directories
study_name = 'Main-study'
data_dir = '/Users/yeganeh/Documents/Raw Files/PLB_HYP_OTKA/Live Sessions'
bids_root = f'/Users/yeganeh/Codes/otka-preprocessing/data/{study_name}'
docs_dir = '/Users/yeganeh/Codes/otka-preprocessing/'

In [ ]:
# creat bids path for the exsiting files
bids_list = []
subjects = ['01','02','03','22']
descriptions = ['baseline1', 'baseline2',
'experience1', 'experience2', 'experience3', 'experience4',
'induction1', 'induction2', 'induction3', 'induction4']

for subject in subjects:
    for desc in descriptions:
        bids_path = BIDSPath(subject=subject,
                            session='01',
                            task= desc,
                            root=bids_root)
        bids_list.append(bids_path)

In [ ]:
# mark bad channels
# read ids map that contains the list of bad channels
ids_map = pd.read_excel(f'{docs_dir}/ids_map.xlsx', header=1)
bad_channels = ids_map[['bids_id','bad_channels']]
bad_channels.set_index('bids_id', inplace=True)
bad_channels = bad_channels.loc[26:46]

# get the bids path corresponding to each participants and mark bad channels for them based on their original ids
value = -1
for ind in bad_channels.index:
    value = value + 1
    temp_list = bids_list[value*10 : (value+1)*10]
    bads = bad_channels.loc[ind][0]
    if bads != '[]':
        for bids_path in temp_list:
            mark_bad_channels(ch_names=ast.literal_eval(bads), bids_path=bids_path, verbose=False, overwrite=True)

# raw = read_raw_bids(bids_path=bids_path, verbose=False)

In [ ]:
# checking the durations in the bids files
# subjects = ['01','02','03','22']
subjects = range(26,51)
descriptions = ['baseline1', 'baseline2',
'experience1', 'experience2', 'experience3', 'experience4',
'induction1', 'induction2', 'induction3', 'induction4']

all_subs_durations = {}

for subject in subjects:
    sub_path = []
    durations = []
    for desc in descriptions:
        bids_path = BIDSPath(subject=str(subject),
                            session='01',
                            task= desc,
                            root=bids_root)
        sub_path.append(bids_path)

    for path in sub_path:                  
        raw = read_raw_bids(path)
        durations.append(raw.annotations.duration)
    
    all_subs_durations[subject] = durations    

pd.set_option('display.max_columns', None)
pd.DataFrame.from_dict(all_subs_durations)   